<a href="https://colab.research.google.com/github/JaySanthanam/Programming-for-data/blob/main/Worksheets/15_2_Decision_Tree_coded_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding a simple decision tree
---

In this worksheet we are going to work with a data set, using the idea of a decision tree class.  We are going to simplify the model and use Python code to make a simple decision tree classification model.  We will do this for two reasons:
*   writing the code is often good for helping to understand what is going on under the bonnet of a library function
*   it is a good coding exercise for practice as it mostly depends on calculations and if..elif..else statements

In this worksheet we are going to code a decision tree which will use the calculated probabilities to make decisions about whether a row of given data would be classified as Iris-virginica, or not, based on sepal and petal dimensions.  It is easier to classify between two values (Iris-virginica or not).  Later, using this information, species would be further predicted by probabilities of error.

![Iris-petals and sepals](https://www.math.umd.edu/~petersd/666/html/iris_with_labels.jpg)

The workflow is:
*  divide the data set into 70% of the rows for training and 30% for testing  (we can increase the size of the training set later)
*  find the median for each of the 4 size columns
*  calculate the proportion of each column that are on or above median that are of a species (ie proportion of petal-lengths on or above median that are Iris-virginica)
*  infer the proportion of each that are not of that species (using 1 - proportion above).  In both cases we are looking to find if either of these is 1, which could be infered as definitely not that species. 
*  calculate a Gini Index that will indicate the probability that a prediction will be incorrect
*  use the results of the Gini Index to model a decision tree
*  code the decision tree model into a function that will return whether or not a row in the test set is predicted to be of species Iris-virginica
*  use the decision tree function to predict, for each row in the test set, if the species will be Iris-virginicia or not, using a set of nested if statements to classify
*  compare the predicted values against the actual values in the test set - what proportion were predicted correctly?


### Exercise 1 - investigate the iris data set
---
Let's start by looking at the data.  We are going to use a data set that contains data on iris flowers.

Read the data at this location: https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv into a dataframe called iris_data

The columns in the CSV file do not have headings, when you read the file, add column headings like this:
```
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, name=names)
```
*  Take a look at the column info (how many columns, what type of data, any missing data?)
*  Take a look at the data values in the first 10 and the last 10 records to get an idea of the type of values included
*  Find out how many unique values there are in the species column
*  Find out the maximum, minimum, median and upper and lower quartile values in each of the columns


In [44]:
import pandas as pd
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, names=names)
iris_data

,sepal-length,sepal-width,petal-length,petal-width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


### Exercise 2 - split the data into train and test sets
---

Split the data set into and 70% train, 30% test, split.  From now, just use the train data set.


In [45]:
# import the train_test_split function
from sklearn.model_selection import train_test_split

# create the classification variables from the all columns
train_orig, test = train_test_split(iris_data, test_size=0.30)
train_orig.describe()

,sepal-length,sepal-width,petal-length,petal-width
count,105.000000,105.000000,105.000000,105.000000
mean,5.767619,3.092381,3.574286,1.138095
std,0.818241,0.428256,1.800642,0.807939
min,4.300000,2.000000,1.100000,0.100000
25%,5.100000,2.800000,1.500000,0.200000
50%,5.700000,3.100000,4.100000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.700000,4.400000,6.700000,2.500000


### Exercise 3 - assumptions and classification
---

Let's make some assumptions based on the data

1.  Iris-setosa, Iris-versicolor, Iris-virginica are the full range of types of iris to be analysed
2.  Although this is a small data set, the means are fairly representative

With these in mind, let's start by classifying sepal/petal size into long/short and wide/narrow with values on or above the mean taken as long or wide and those below as short or narrow.

This is a starting point.  We will be trying to find a value (indicator) for each column where rows on or above do not contain any of a particular species, this might indicate that this column is a good (if not rough) indicator of species.

*  Drop any null values from each column

Calculate, and store the means of the four columns

*  **Test**:
Display train.describe() to see the value of the means of the training set. Print the four means and compare to the output of train.describe() to check that they have been calculated correctly.

*  Create a new dataframe with the numeric columns encoded so show a 1 for any value that is above the mean for its column and 0 for any that isn't.




In [46]:
# get the mean for each column and apply a function to encode into 1 (above mean) and 0 (mean or below mean)
def encode(df, **kwds):
  cutoff = kwds['indicator']
  key = kwds['key']
  if df[key] > cutoff:
    return 1
  else:
    return 0


# run the function for each column so that each of the four columns are encoded, then drop the original columns, saving as a new dataframe
train['sepal-length'] = train_orig.apply(encode, axis=1, key='sepal-length', indicator=train_orig['sepal-length'].mean())
train['sepal-width'] = train_orig.apply(encode, axis=1, key='sepal-width', indicator=train_orig['sepal-width'].mean())
train['petal-length'] = train_orig.apply(encode, axis=1, key='petal-length', indicator=train_orig['petal-length'].mean())
train['petal-width'] = train_orig.apply(encode, axis=1, key='petal-width', indicator=train_orig['petal-width'].mean())

print(train.describe())


       sepal-length  sepal-width  petal-length  petal-width
count     73.000000    73.000000     73.000000    73.000000
mean       0.465753     0.534247      0.589041     0.547945
std        0.502278     0.502278      0.495413     0.501140
min        0.000000     0.000000      0.000000     0.000000
25%        0.000000     0.000000      0.000000     0.000000
50%        0.000000     1.000000      1.000000     1.000000
75%        1.000000     1.000000      1.000000     1.000000
max        1.000000     1.000000      1.000000     1.000000


### Exercise 4 - Calculate the proportion of values on or above the mean that are of each species

We are going to focus on the `Iris-virginica` species first.

First we will calculate, for each dimension column (`sepal-length, sepal-width, petal-length, petal-width`) what proportion of values in that column, where the value is on or above the mean, are classified as `Iris-virginica`.

We will do this by filtering all the records in each column of the the `train` set that are on or above the mean and match the species .  Then use the outcome to calculate the proportion of the full `train` set for which a value on or above the mean that are of species `Iris-virginica`.

*  filter for values in the `sepal-length` column being on or above the mean and the species column being `Iris-virginica`.  Then divide the count of rows in this filtered dataset by the count of rows in a second data set, filtered for just the value being on or above the mean.

*  Do this for all four columns, for `Iris-virginica`  (4 operations).

Print the results to see which columns look like they might most reliably predict the species as `Iris-virginica` (the result is as close as possible to 1).  The highest numbers may indicate the most reliable indicators, but we will do some more before coming to this conclusion.

*  By definition, those on or above the mean that are NOT Iris_virginica will be `1 - the proportion of those that are.  Calculate these

The first one has been done for you.

*  We will also need the proportion of those BELOW the median that are NOT Iris-virginica.  Calculate these in the same way



In [47]:
# calculate the proportion of results where the value is on or above mean that are of the species Iris-virginica
def get_prop_above(df, spec,column):
  train_spec = df[(df[column]== 1) & (df['species']==spec)][column].count()
  train_all = df[df[column]== 1][column].count()
  prop_spec = train_spec/train_all
  return prop_spec

vir_sep_len = get_prop_above(train, 'Iris-virginica', 'sepal-length')
vir_sep_width = get_prop_above(train, 'Iris-virginica', 'sepal-width')
vir_petal_len = get_prop_above(train, 'Iris-virginica', 'petal-length')
vir_petal_width = get_prop_above(train, 'Iris-virginica', 'petal-width')

print("Proportion using Sepal length using above mean: ", vir_sep_len, "\nProportion using Sepal width using above mean: ",vir_sep_width, "\nProportion using Petal length using above mean: ",vir_petal_len, "\nProportion using Petal width using above mean: ", vir_petal_width)

Proportion using Sepal length using above mean:  0.7058823529411765 
Proportion using Sepal width using above mean:  0.358974358974359 
Proportion using Petal length using above mean:  0.5813953488372093 
Proportion using Petal width using above mean:  0.625


From the proportion test, we see none of the numbers are anywhere near 1. Particularly, sepal width shows that very small proportion of Iris-virginica has sepal-width above or equal to mean.

### Exercise 5 - Calculate the proportion of each column where the value is below median that are of species `Iris-virginica`

Repeat the code above, this time looking for values below the mean

In [48]:
# calculate the proportion of results where the value is below mean that are of the species Iris-virginica
def get_prop_below(df, spec,column):
  train_spec = df[(df[column]== 0) & (df['species']==spec)][column].count()
  train_all = df[df[column]== 0][column].count()
  prop_spec = train_spec/train_all
  return prop_spec

vir_sep_len1 = get_prop_below(train, 'Iris-virginica', 'sepal-length')
vir_sep_width1 = get_prop_below(train, 'Iris-virginica', 'sepal-width')
vir_petal_len1 = get_prop_below(train, 'Iris-virginica', 'petal-length')
vir_petal_width1 = get_prop_below(train, 'Iris-virginica', 'petal-width')

print("Proportion using Sepal length using below mean: ", vir_sep_len1, "\nProportion using Sepal width using below mean: ",vir_sep_width1, "\nProportion using Petal length using below mean: ",vir_petal_len1, "\nProportion using Petal width using below mean: ", vir_petal_width1)

Proportion using Sepal length using below mean:  0.02564102564102564 
Proportion using Sepal width using below mean:  0.3235294117647059 
Proportion using Petal length using below mean:  0.0 
Proportion using Petal width using below mean:  0.0


### Exercise 5 - calculate for the other two Iris species
---

Do the same calculations for the Iris-versicolor species, then for the Iris-setosa species.






In [49]:
# Iris-setosa

set_sep_len = get_prop_above(train, 'Iris-setosa', 'sepal-length')
set_sep_width = get_prop_above(train, 'Iris-setosa', 'sepal-width')
set_petal_len = get_prop_above(train, 'Iris-setosa', 'petal-length')
set_petal_width = get_prop_above(train, 'Iris-setosa', 'petal-width')

set_sep_len1 = get_prop_below(train, 'Iris-setosa', 'sepal-length')
set_sep_width1 = get_prop_below(train, 'Iris-setosa', 'sepal-width')
set_petal_len1 = get_prop_below(train, 'Iris-setosa', 'petal-length')
set_petal_width1 = get_prop_below(train, 'Iris-setosa', 'petal-width')

#Iris-versicolor

ver_sep_len = get_prop_above(train, 'Iris-versicolor', 'sepal-length')
ver_sep_width = get_prop_above(train, 'Iris-versicolor', 'sepal-width')
ver_petal_len = get_prop_above(train, 'Iris-versicolor', 'petal-length')
ver_petal_width = get_prop_above(train, 'Iris-versicolor', 'petal-width')

ver_sep_len1 = get_prop_below(train, 'Iris-versicolor', 'sepal-length')
ver_sep_width1 = get_prop_below(train, 'Iris-versicolor', 'sepal-width')
ver_petal_len1 = get_prop_below(train, 'Iris-versicolor', 'petal-length')
ver_petal_width1 = get_prop_below(train, 'Iris-versicolor', 'petal-width')

### Exercise 6 - predict from the results
---

Create a list of dictionaries from the results Exercise 4 and 5 (e.g. {'species':..., 'above_mean': 0.xx, 'below_mean': 0.xx}  

Then use a loop to go through the list and print:  
*  any species and indicator (above or below mean) that can reliably be predicted.  A reliable prediction may be one over 0.5

In [50]:
#Create dictionary from keys and values

keys_list = ["species", "sepal length above mean", "sepal length below mean", "sepal width above mean", "sepal width below mean", "petal length above mean", "petal length below mean", "petal width above mean", "petal width below mean"]
species_list = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
sep_len_ab_mean = [set_sep_len, ver_sep_len, vir_sep_len]
sep_wid_ab_mean = [set_sep_width, ver_sep_width, vir_sep_width]
pet_len_ab_mean = [set_petal_len, ver_petal_len, vir_sep_len]
pet_wid_ab_mean = [set_petal_width, ver_petal_width, vir_petal_width]
sep_len_bel_mean = [set_sep_len1, ver_sep_len1, vir_sep_len1]
sep_wid_bel_mean = [set_sep_width1, ver_sep_width1, vir_sep_width1]
pet_len_bel_mean = [set_petal_len1, ver_petal_len1, vir_sep_len1]
pet_wid_bel_mean = [set_petal_width1, ver_petal_width1, vir_petal_width1]

zip_iterator = zip(keys_list, species_list, sep_len_ab_mean)

iris_dict = dict(zip_iterator)

iris_dict

ValueError: ignored

I can only use 2 lists to create a dictionary (unless I create a dictionary of lists. This means that I will have create a dictionaly for each species. I am not sure if it make sense to have multiple dictionaries - unless I see a task below using these dictionaries. So I will create a dataframe instead so I can look at the species and the parameter values in one shot.

In [51]:
#create a dataframe to see the results

iris_df = pd.DataFrame(list(zip(species_list, sep_len_ab_mean, sep_wid_ab_mean, pet_len_ab_mean, pet_wid_ab_mean, sep_len_bel_mean, sep_wid_bel_mean, pet_len_bel_mean, pet_wid_bel_mean)), columns =['species', 'sepal_length_above_mean','sepal_width_above_mean','petal_length_above_mean','petal_width_above_mean','sepal_length_below_mean','sepal_width_below_mean','petal_length_below_mean','petal_width_below_mean'])
iris_df

# show which columns are reliable predictors



,species,sepal_length_above_mean,sepal_width_above_mean,petal_length_above_mean,petal_width_above_mean,sepal_length_below_mean,sepal_width_below_mean,petal_length_below_mean,petal_width_below_mean
0,Iris-setosa,0.000000,0.589744,0.000000,0.000,0.743590,0.176471,0.966667,0.878788
1,Iris-versicolor,0.294118,0.051282,0.418605,0.375,0.230769,0.500000,0.033333,0.121212
2,Iris-virginica,0.705882,0.358974,0.705882,0.625,0.025641,0.323529,0.025641,0.000000


1. Iris-setosa clearly has smaller than average sepal length (0.64), petal length (0.87) and petal width (0.79).
2. Iris-virginica has larger than average sepal length (0.65), petal length (0.65) and petal width (0.61).
3. Iris-versicolor is around average in all columns. 

To me, it looks like petal length and width are really good way identify the species. So something along the lines of...if petal length and petal width are smaller than average then the species is Iris-setosa. If on the other hand they both are larger than average, the species is Iris-virginica. otherwise, it is Iris-versicolor

Looking at the above results, I think the sepal width both above and below mean seem to be not very reliabale/consistent meaure to classify the species. Also are the length and width of the sepals and petals correlated? That is if the length is more is the width smaller (thin and long?)or they are unconnected. This made me wonder - what if I did a correlation test to see how the sepal length and width are correlated to petal length and width. So here is a heatmap using seaborn.

In [ ]:
import seaborn as sns
sns.heatmap(train_orig.corr(), annot = True);

So this shows that the sepal width is most likely independent of the other variables while petal length and width seem to be highly correlated. So maybe as a first test petal length and width might be good indicators to find the species. Then we can add sepal length to further confirm the result.

### Exercise 6 - Make a decision tree
---

Use pencil and paper or a graphical application to create a decision tree for Iris-virginica, using the following rules (use the picture below as a guide):

*  The column with the highest indicator is placed at the top
*  Other columns are placed in order below
*  The rest of the columns are placed in order below these

Any column where one branch (on or above mean OR below mean) has an indicator of 0, could be classified as a strong indicator of Iris_virginica being the species.  Anything else, unless there is something very close to 0, could be classified as a weak indicator of Iris_virginica being the species.

Let's code the decision tree using the following logic for this decision tree (yours might be slightly different):

![Decision tree](https://drive.google.com/uc?id=1CTo23EHwR2IPCRjcfSyCQsT_oQ5Exwso)

In the decision tree above, there is no certainty below petal-length so our decision tree will only include petal-width and petal-length.




In [ ]:
def get_species(df):
  # ADD CODE HERE TO RETURN None if petal-width is below mean (encoded as 0) or if petal-length is below mean (encoded as 0), otherwise return 'Iris-virginia'
    if df['petal-width'] == 0:
        return None
        if df['petal-length'] == 0:
            return None
            if df['sepal-length'] == 0:
                return None
    return 'Iris-virginica' 

# use the get_species(df) function to predict the species, count how many are predicted correct and use this to calculate the proportion correct
correct = 0
test_size = test.shape[0]
for i in range(0, test_size):
  species = get_species(test.iloc[i])
  if species == test.iloc[i]['species']:
      correct += 1

print ("Proportion correctly identified", correct / test_size) 


### Exercise 7 - change the measure

We are currently using the mean to act as the decision making line.  We can use the decision tree with a different line.

Change the mean values so that you are instead using the median instead for all four columns.  The code should not need changing except for where you calculated the mean.

Run all the code again.  Is the proportion of correct values better this time?   Is the decision tree still appropriate?


What do you notice? (write your answer here)

### Exercise 8 - try different measures
---

Do the same again but with upper quantile, then again with the lower quantile.  Is it making any difference?  Which give the best looking results?

### Exercise 9 - try a different species

Run the mean test again for the Iris-versicolor species.  Again, try some different decision making lines.

What are the results.  Record them in the text cell below:

Write your answers here:  

# New logic introduced in this worksheet:

1.  Adding headings to a CSV if none currently exist
2.  Splitting a data set into train and test sets

In [ ]:
## this type of plot will show the distribution on a chart
from plotnine import *
ggplot(train, aes(x='petal-length', y='petal-width', color='species')) + geom_point() + geom_vline(train, aes(xintercept=train['sepal-length'].mean(), color='species')) + geom_hline(train, aes(yintercept=train['sepal-width'].mean(), color='species'))